<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/toNamedTuple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/dotmap.ipynb
!pip install importnb
from importnb import Notebook
with Notebook(): 
    from dotmap import DotMap
from collections import namedtuple
import jax
import numpy as np
import jax.numpy as jnp
import jax.tree_util as tu
from functools import partial
from copy import deepcopy

In [ ]:
def recurse(d,e={},root='r'):
    for k,v in d.items():
        if isinstance(v,dict):
            e[k]={}
            e[k]=recurse(v,e[k],root+"_"+k)
        else:
            e[k]=v
    NT=namedtuple(root,e.keys())
    return NT(**e)

In [ ]:
def toNamedTuple(orig):
    d=deepcopy(orig)
    stack=[(d,dictHandler(d,'toNamedTupleResult'))]


    while stack:
        nodict=True
        source,setter=stack[-1]
        for k,v in source.items():
            if isinstance(v,dict):
                stack.append((v, dictHandler(source,k)))
                nodict=False
        if nodict:            
            setter(source)
            stack.pop()
            
    return d['toNamedTupleResult']

In [11]:

def dictHandler(build,k):
    def dictSetter(v):
        if isinstance(v,dict):
            NT = namedtuple('_', v.keys())
            build[k]=NT(**v)
        elif isinstance(v,list):
            build[k]=tuple(v)
        return
    return dictSetter

def listHandler(l,i):
    def listSetter(v):
        if isinstance(v,list):
            l[i]=tuple(v)
        elif isinstance(v,dict):
            NT = namedtuple('_', v.keys())
            l[i]=NT(**v)
        else:
            l[i]=v
        return
    return listSetter

def toNamedTuple3(orig):
    d=deepcopy(orig)

    def listRoot(v):
        nonlocal d
        d=tuple(v)


    def dictRoot(v):
        nonlocal d
        NT = namedtuple('_', v.keys())
        d=NT(**v)

    if isinstance(d,list):
        stack=[(d, listRoot)]
    elif isinstance(d, dict):
        # stack=[(d, dictHandler(d,'result'))]
        stack=[(d, dictRoot)]

    count=0
    while stack:
        nested=False
        source,setter=stack[-1]
        # printlist(stack)
        # print(source)
        # print(d)
        # print('---')
        if isinstance(source,list):
            for i,v in enumerate(source):
                if type(v) in (dict,list):
                    stack.append((v, listHandler(source,i)))
                    nested=True

        elif isinstance(source,dict):
            for k,v in source.items():
                if type(v) in (dict,list):
                    stack.append((v, dictHandler(source,k)))
                    nested=True

        if not nested: 
            setter(source)
            stack.pop()
        count+=1
    return d

In [15]:
d=dict(b=dict(m=2,n=4))
e=dict(c=1,g=dict(z=2))
f=dict(x=3.,y=[d,d])

z=[dict(a=d, k=[d,[e,f]]), f]
dt=toNamedTuple3(z)
dt

(_(a=_(b=_(m=2, n=4)), k=(_(b=_(m=2, n=4)), (_(c=1, g=_(z=2)), _(x=3.0, y=(_(b=_(m=2, n=4)), _(b=_(m=2, n=4))))))),
 _(x=3.0, y=(_(b=_(m=2, n=4)), _(b=_(m=2, n=4)))))

In [16]:
val,tree = tu.tree_flatten(dt)

In [17]:
val,tree

([2, 4, 2, 4, 1, 2, 3.0, 2, 4, 2, 4, 3.0, 2, 4, 2, 4],
 PyTreeDef((CustomNode(namedtuple[<class '__main__._'>], [CustomNode(namedtuple[<class '__main__._'>], [CustomNode(namedtuple[<class '__main__._'>], [*, *])]), (CustomNode(namedtuple[<class '__main__._'>], [CustomNode(namedtuple[<class '__main__._'>], [*, *])]), (CustomNode(namedtuple[<class '__main__._'>], [*, CustomNode(namedtuple[<class '__main__._'>], [*])]), CustomNode(namedtuple[<class '__main__._'>], [*, (CustomNode(namedtuple[<class '__main__._'>], [CustomNode(namedtuple[<class '__main__._'>], [*, *])]), CustomNode(namedtuple[<class '__main__._'>], [CustomNode(namedtuple[<class '__main__._'>], [*, *])]))])))]), CustomNode(namedtuple[<class '__main__._'>], [*, (CustomNode(namedtuple[<class '__main__._'>], [CustomNode(namedtuple[<class '__main__._'>], [*, *])]), CustomNode(namedtuple[<class '__main__._'>], [CustomNode(namedtuple[<class '__main__._'>], [*, *])]))]))))

In [18]:
val

[2, 4, 2, 4, 1, 2, 3.0, 2, 4, 2, 4, 3.0, 2, 4, 2, 4]

In [19]:
val2 = np.random.normal(size=len(val))

In [20]:
%%timeit
tu.tree_unflatten(tree, val2)

The slowest run took 6.25 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 13.1 µs per loop
